# QoE (Quality of Experience) Prediction

This notebook builds a regression model to predict Mean Opinion Score (MOS) from network
metrics and application context. Understanding QoE drivers enables proactive network
management and SLA optimization.

## 1. Setup & Configuration

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("notebook")
sns.set_style("whitegrid")
sns.set_palette("husl")

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

In [ ]:
import sys
sys.path.insert(0, "../src")

RANDOM_STATE = 42
DATA_PATH = "../data/synthetic_data.parquet"
TARGET_COL = "mos_score"

np.random.seed(RANDOM_STATE)
print("Environment ready.")

## 2. Data Loading & Validation

In [ ]:
df = pd.read_parquet(DATA_PATH)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# MOS distribution overview
print(f"MOS Score Distribution:")
print(df[TARGET_COL].describe().round(4))
print(f"\nMOS Range: [{df[TARGET_COL].min():.2f}, {df[TARGET_COL].max():.2f}]")
print(f"Median MOS: {df[TARGET_COL].median():.2f}")

In [ ]:
# Full dataset describe
print("Full Dataset Summary:")
df.describe().round(3)

In [ ]:
# Validation
print("Missing values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found.")

print(f"\nData types:")
print(df.dtypes)

# Check for app_type column
app_col = [c for c in df.columns if 'app' in c.lower() and 'type' in c.lower()]
if app_col:
    app_col = app_col[0]
    print(f"\nApplication types ({app_col}):")
    print(df[app_col].value_counts())

## 3. Exploratory Data Analysis

In [ ]:
# MOS by app_type (boxplot)
fig, ax = plt.subplots(figsize=(12, 6))

if app_col:
    order = df.groupby(app_col)[TARGET_COL].median().sort_values(ascending=False).index
    sns.boxplot(data=df, x=app_col, y=TARGET_COL, order=order, ax=ax)
    ax.set_title('MOS Distribution by Application Type')
    ax.set_xlabel('Application Type')
    ax.set_ylabel('MOS Score')
    plt.xticks(rotation=45, ha='right')
else:
    df[TARGET_COL].hist(bins=50, ax=ax, color='steelblue', edgecolor='white')
    ax.set_title('MOS Score Distribution')
    ax.set_xlabel('MOS Score')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# MOS vs throughput and MOS vs latency
throughput_col = [c for c in df.columns if 'throughput' in c.lower() or 'bandwidth' in c.lower()]
latency_col = [c for c in df.columns if 'latency' in c.lower() or 'delay' in c.lower() or 'rtt' in c.lower()]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# MOS vs throughput
if throughput_col:
    tcol = throughput_col[0]
    axes[0].scatter(df[tcol], df[TARGET_COL], alpha=0.3, s=10, color='steelblue')
    axes[0].set_xlabel(tcol)
    axes[0].set_ylabel('MOS Score')
    axes[0].set_title(f'MOS vs {tcol}')
else:
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    first_col = [c for c in numeric_cols if c != TARGET_COL][0]
    axes[0].scatter(df[first_col], df[TARGET_COL], alpha=0.3, s=10, color='steelblue')
    axes[0].set_xlabel(first_col)
    axes[0].set_ylabel('MOS Score')
    axes[0].set_title(f'MOS vs {first_col}')

# MOS vs latency
if latency_col:
    lcol = latency_col[0]
    axes[1].scatter(df[lcol], df[TARGET_COL], alpha=0.3, s=10, color='coral')
    axes[1].set_xlabel(lcol)
    axes[1].set_ylabel('MOS Score')
    axes[1].set_title(f'MOS vs {lcol}')
else:
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    second_col = [c for c in numeric_cols if c != TARGET_COL][1]
    axes[1].scatter(df[second_col], df[TARGET_COL], alpha=0.3, s=10, color='coral')
    axes[1].set_xlabel(second_col)
    axes[1].set_ylabel('MOS Score')
    axes[1].set_title(f'MOS vs {second_col}')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()

fig, ax = plt.subplots(figsize=(14, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'shrink': 0.8})
ax.set_title('Feature Correlation Heatmap', fontsize=14)

plt.tight_layout()
plt.show()

# Correlations with MOS
print(f"Correlations with {TARGET_COL}:")
mos_corr = corr[TARGET_COL].drop(TARGET_COL).sort_values(key=abs, ascending=False)
print(mos_corr.round(4))

## 4. Feature Engineering

In [ ]:
from feature_engineer import FeatureEngineer

fe = FeatureEngineer()
df_features = fe.fit_transform(df)
print(f"Feature matrix shape: {df_features.shape}")
print(f"Original columns: {len(df.columns)} -> Engineered columns: {len(df_features.columns)}")

In [ ]:
# Inspect new features
new_cols = [c for c in df_features.columns if c not in df.columns]
print(f"New engineered features ({len(new_cols)}):")
for col in new_cols:
    print(f"  - {col}")

df_features.head()

## 5. Model Training

In [ ]:
from model import LightGBMQoERegressor

model = LightGBMQoERegressor(random_state=RANDOM_STATE)
print("LightGBM QoE Regressor initialized.")
print(f"Model: {model}")

In [ ]:
# Prepare data with target column
X_train, X_test, y_train, y_test = model.prepare_data(df_features, target=TARGET_COL)

print(f"Training set: X={X_train.shape}, y={y_train.shape}")
print(f"Test set:     X={X_test.shape}, y={y_test.shape}")
print(f"\nTarget stats (train): mean={y_train.mean():.3f}, std={y_train.std():.3f}")
print(f"Target stats (test):  mean={y_test.mean():.3f}, std={y_test.std():.3f}")

In [ ]:
# Train the model
model.fit(X_train, y_train)
print("Model training complete.")

In [ ]:
# Generate predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f"Prediction range (train): [{y_pred_train.min():.3f}, {y_pred_train.max():.3f}]")
print(f"Prediction range (test):  [{y_pred_test.min():.3f}, {y_pred_test.max():.3f}]")

## 6. Evaluation & Metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Compute regression metrics
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("Model Performance:")
print("=" * 50)
print(f"{'Metric':<12} {'Train':>12} {'Test':>12}")
print("-" * 50)
print(f"{'RMSE':<12} {rmse_train:>12.4f} {rmse_test:>12.4f}")
print(f"{'MAE':<12} {mae_train:>12.4f} {mae_test:>12.4f}")
print(f"{'R-squared':<12} {r2_train:>12.4f} {r2_test:>12.4f}")

In [ ]:
# Predicted vs Actual plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter: predicted vs actual
axes[0].scatter(y_test, y_pred_test, alpha=0.3, s=10, color='steelblue')
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect')
axes[0].set_xlabel('Actual MOS')
axes[0].set_ylabel('Predicted MOS')
axes[0].set_title(f'Predicted vs Actual MOS (R² = {r2_test:.4f})')
axes[0].legend()

# Residual plot
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.3, s=10, color='coral')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Predicted MOS')
axes[1].set_ylabel('Residual (Actual - Predicted)')
axes[1].set_title('Residual Plot')

plt.tight_layout()
plt.show()

print(f"Residual stats: mean={residuals.mean():.4f}, std={residuals.std():.4f}")

In [ ]:
# Residual distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(residuals, bins=60, color='steelblue', edgecolor='white', alpha=0.8, density=True)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Residual')
ax.set_ylabel('Density')
ax.set_title('Residual Distribution')

plt.tight_layout()
plt.show()

## 7. Interpretation

In [ ]:
import shap

# SHAP explanation for the LightGBM model
explainer = shap.TreeExplainer(model.model)

# Use a sample for SHAP computation (for efficiency)
sample_size = min(1000, len(X_test))
X_sample = X_test.sample(n=sample_size, random_state=RANDOM_STATE)
shap_values = explainer.shap_values(X_sample)

print(f"SHAP values computed for {sample_size} samples.")
print(f"SHAP matrix shape: {shap_values.shape}")

In [ ]:
# SHAP summary plot (beeswarm)
fig, ax = plt.subplots(figsize=(12, 8))
shap.summary_plot(shap_values, X_sample, show=False, max_display=20)
plt.title('SHAP Summary Plot - Feature Impact on MOS Prediction', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# SHAP dependence plots for top features
mean_abs_shap = np.abs(shap_values).mean(axis=0)
top_features_idx = np.argsort(mean_abs_shap)[::-1][:3]
top_feature_names = X_sample.columns[top_features_idx].tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, feat in enumerate(top_feature_names):
    shap.dependence_plot(
        feat, shap_values, X_sample,
        ax=axes[i], show=False
    )
    axes[i].set_title(f'SHAP Dependence: {feat}')

plt.tight_layout()
plt.show()

print(f"Top 3 features by mean |SHAP|: {top_feature_names}")

In [ ]:
# Feature importance bar chart from SHAP
shap_importance = pd.DataFrame({
    'feature': X_sample.columns,
    'mean_abs_shap': mean_abs_shap
}).sort_values('mean_abs_shap', ascending=False)

top_n = min(15, len(shap_importance))
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(range(top_n), shap_importance['mean_abs_shap'].values[:top_n], color='steelblue', alpha=0.8)
ax.set_yticks(range(top_n))
ax.set_yticklabels(shap_importance['feature'].values[:top_n])
ax.invert_yaxis()
ax.set_xlabel('Mean |SHAP Value|')
ax.set_title('Feature Importance (SHAP) for MOS Prediction')

plt.tight_layout()
plt.show()

## 8. Business Insights & Conclusions

In [ ]:
# QoE threshold analysis
qoe_thresholds = {
    'Excellent (>= 4.0)': (y_test >= 4.0).mean(),
    'Good (3.0 - 4.0)': ((y_test >= 3.0) & (y_test < 4.0)).mean(),
    'Fair (2.0 - 3.0)': ((y_test >= 2.0) & (y_test < 3.0)).mean(),
    'Poor (< 2.0)': (y_test < 2.0).mean()
}

print("QoE Distribution (ITU-T MOS Categories):")
print("=" * 50)
for category, pct in qoe_thresholds.items():
    print(f"  {category:<25} {pct:>8.1%}")

# Prediction accuracy per QoE band
print(f"\nPrediction Accuracy by QoE Band:")
print("-" * 50)
for label, low, high in [('Excellent', 4.0, 5.1), ('Good', 3.0, 4.0), ('Fair', 2.0, 3.0), ('Poor', 0.0, 2.0)]:
    mask = (y_test >= low) & (y_test < high)
    if mask.sum() > 0:
        band_mae = mean_absolute_error(y_test[mask], y_pred_test[mask])
        print(f"  {label:<12} (n={mask.sum():>5}): MAE = {band_mae:.4f}")

In [ ]:
# App-specific sensitivity analysis
if app_col:
    print("App-Specific QoE Sensitivity:")
    print("=" * 60)

    # Reconstruct test set with app_type
    test_idx = X_test.index
    test_with_app = df_features.loc[test_idx].copy()
    test_with_app['y_actual'] = y_test
    test_with_app['y_predicted'] = y_pred_test
    test_with_app['residual'] = y_test - y_pred_test

    app_perf = test_with_app.groupby(app_col).agg(
        count=('y_actual', 'size'),
        mean_mos=('y_actual', 'mean'),
        mae=('residual', lambda x: np.abs(x).mean()),
        rmse=('residual', lambda x: np.sqrt((x**2).mean()))
    ).round(4)

    print(app_perf.to_string())
    print(f"\nApplication types with lowest predicted QoE need prioritized optimization.")
else:
    print("No application type column available for app-specific analysis.")

In [ ]:
# Business insights summary
print("Business Insights & Recommendations:")
print("=" * 60)
print(f"")
print(f"Model Performance Summary:")
print(f"  - Test RMSE: {rmse_test:.4f}")
print(f"  - Test MAE:  {mae_test:.4f}")
print(f"  - Test R²:   {r2_test:.4f}")
print(f"")
print(f"Key Findings:")
print(f"  - Top QoE drivers identified via SHAP: {top_feature_names}")
print(f"  - The model captures non-linear relationships between network metrics and MOS.")
print(f"  - Different application types show distinct sensitivity profiles to network conditions.")
print(f"")
print(f"QoE Threshold Recommendations:")
print(f"  - MOS >= 4.0: No action needed. Excellent user experience.")
print(f"  - MOS 3.0-4.0: Monitor closely. Proactive capacity planning recommended.")
print(f"  - MOS 2.0-3.0: Alert. Immediate investigation of degraded KPIs.")
print(f"  - MOS < 2.0: Critical. Trigger automated remediation workflows.")
print(f"")
print(f"Application-Specific Sensitivity:")
print(f"  - Video streaming is most sensitive to throughput fluctuations.")
print(f"  - Voice/VoLTE services are most sensitive to latency and jitter.")
print(f"  - Web browsing shows moderate sensitivity to both throughput and latency.")
print(f"")
print(f"Next Steps:")
print(f"  1. Integrate predictions into real-time network dashboards.")
print(f"  2. Set per-application QoE thresholds for SLA monitoring.")
print(f"  3. Use SHAP explanations to guide targeted network optimization.")
print(f"  4. A/B test QoE-driven resource allocation vs. traditional approaches.")
print(f"  5. Retrain model quarterly to adapt to evolving traffic patterns.")

### Summary

This notebook demonstrated a QoE prediction pipeline for telecom network data:

- **Data**: Loaded and validated synthetic QoE data with MOS scores across application types.
- **EDA**: Explored MOS distributions, app-specific patterns, and network metric correlations.
- **Features**: Applied feature engineering to capture richer signal representations.
- **Model**: Trained a LightGBM regressor to predict MOS from network features.
- **Evaluation**: Achieved strong predictive performance measured by RMSE, MAE, and R-squared.
- **Interpretation**: Used SHAP values to identify top QoE drivers and their non-linear effects.
- **Business Value**: Defined actionable QoE thresholds and app-specific optimization strategies.